In [13]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [14]:
from weather_api import *

In [12]:
df = get_data("zakopane")
df
#Jak widać kolumny dają nam za mało informacji, trzeba obgadac które dane wyciagnac

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Status pomiaru NOS,Średnia dobowa prędkość wiatru [m/s],Status pomiaru FWS,Średnia dobowa temperatura [°C],...,Średnia dobowa wilgotność względna [%],Status pomiaru WLGS,Średnia dobowe ciśnienie na poziomie stacji [hPa],Status pomiaru PPPS,Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Status pomiaru WODZ,Suma opadu noc [mm],Status pomiaru WONO


In [ ]:
__all__ = ["get_data"]

DATA_FOLDER = "./data"

def read_column_names(file):
    with open(file, 'r') as f:
        content = f.readlines()
    
    return [x.strip() for x in content]

def delete_mostly_nan_columns(df):
    column_names = list()
    all_entries = len(df.index)
    nan_values = df.isnull().sum().to_numpy()

    for i in range(len(nan_values)):
        if (nan_values[i]/all_entries)>=0.3:
            column_names.append(df.columns[i])

    for name in column_names:
        if name in df.columns:
            del df[name]

def normalize_particular_columns(df, column_names):
    for col_name in column_names:
        df.iloc[:,col_name]=(df.iloc[:,col_name]-df.iloc[:,col_name].mean())/df.iloc[:,col_name].std()


In [ ]:
columns_names = read_column_names('./data/head.txt')
df1 = pd.read_csv('./data/01_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df2 = pd.read_csv('./data/02_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df3 = pd.read_csv('./data/03_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df4 = pd.read_csv('./data/04_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df5 = pd.read_csv('./data/05_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df6 = pd.read_csv('./data/06_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
delete_mostly_nan_columns(df1)
delete_mostly_nan_columns(df2)
delete_mostly_nan_columns(df3)
delete_mostly_nan_columns(df4)
delete_mostly_nan_columns(df5)
delete_mostly_nan_columns(df6)

data = [df1,df2,df3,df4,df5,df6]
result = pd.concat(data)

In [ ]:
result.head()

In [ ]:
value = []
data = result.iloc[:,[13]]
data = data.to_numpy()
for row in data:
    if row == 0:
        value.append(0)
    elif row > 0:
        value.append(1)
result['deszcz w dzień']= value

value1 = []
data1 = result.iloc[:,[14]]
data1 = data1.to_numpy()
for row1 in data1:
    if row1 == 0:
        value1.append(0)
    elif row1 > 0:
        value1.append(1)
result['deszcz w nocy']= value

In [ ]:
result

In [ ]:
columns_to_normalize = [[5],[6],[8],[9],[10]]
normalize_particular_columns(result, columns_to_normalize)

In [ ]:
result = result.drop(result.columns[11],axis = 1)
result.head()

In [ ]:
X = result[[result.columns[5],result.columns[6],result.columns[7],result.columns[8],result.columns[9],result.columns[10]]]
y = result[[result.columns[14]]]

In [ ]:
Xtrain, Xtest, ytrain, ytest  = train_test_split(X,y,test_size=0.2,random_state=75)

Model SVN 

In [ ]:
clf= svm.SVC(kernel='linear', C=1.0)
clf.fit(Xtrain,ytrain)

In [ ]:
ypred = clf.predict(Xtest)
clf_score = clf.score(Xtest,ytest)

In [ ]:
clf_score

In [ ]:
confusion_matrix(ytest,ypred)

Model logistic regresion

In [ ]:
logistic_regression= LogisticRegression()
logistic_regression.fit(Xtrain,ytrain)
y_pred_r=logistic_regression.predict(Xtest)
    
accur = accuracy_score(ytest, y_pred_r)

In [ ]:
accur

In [ ]:
confusion_matrix(ytest,y_pred_r)

Model KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(Xtrain, ytrain)
y_pred_knn = knn.predict(Xtest)
accurKNN = accuracy_score(ytest, y_pred_knn)

In [ ]:
accurKNN

In [ ]:
confusion_matrix(ytest,y_pred_knn)